In [1]:
import numpy as np
from typing import List, Union

from nomad.metainfo import (
    MSection, Quantity, SubSection, Section, Reference, MEnum
)
from nomad.datamodel.metainfo.basesections import (
    ProcessStep, SynthesisMethod, CompositeSystemReference, InstrumentReference
)


from nomad.datamodel.metainfo.eln import ( Experiment as ELNExperiment, Process as ELNProcess, Sample as ELNSample,)


In [2]:
def add_display_unit_to_kwargs(kwargs):
    """
    Adds a_display with the appropriate unit to a Quantity's kwargs if it has a unit.
    
    Args:
        kwargs: Dictionary of keyword arguments for a Quantity
        
    Returns:
        Modified kwargs dictionary with a_display added if appropriate
    """
    if 'unit' in kwargs:
        if 'a_display' not in kwargs:
            kwargs['a_display'] = {'unit': kwargs['unit']}
        elif 'unit' not in kwargs['a_display']:
            kwargs['a_display']['unit'] = kwargs['unit']
    return kwargs

# Keep a reference to the original Quantity class
original_quantity = Quantity

# Function to create a Quantity with display units
def quantity_with_display(type=None, **kwargs):
    """
    Factory function that creates a Quantity with display units added.
    
    Args:
        type: Type of the quantity
        **kwargs: Keyword arguments for the Quantity
        
    Returns:
        A Quantity with a_display added if it has a unit
    """
    kwargs = add_display_unit_to_kwargs(kwargs)
    return original_quantity(type=type, **kwargs)


In [3]:
class SpinRamp(MSection):
    '''
    Spin ramp profile for spin coating.
    '''

    m_def = Section(
        a_plotly_graph_object=[{
            "data": { "x": "#time", "y": "#speed"},
        }]
    )
    
    speed = quantity_with_display(
        type=int,
        unit='rpm',
        shape=["*"],
        description='Speed of the spin ramp in revolutions per minute.',
        a_eln=dict(component='NumberEditQuantity', default=[1000, 2000, 3000]),
        a_display={ 'unit': 'rpm' }
    )

    time = quantity_with_display(
        type=int,
        unit='second',
        shape=["*"],
        description='Time at each speed in the spin ramp.',
        a_eln=dict(component='NumberEditQuantity', default=[10, 30, 60])
    )

class LinearDispensePattern(MSection):
    '''
    Linear dispensing pattern for spin coating.
    '''

    m_def = Section(a_eln={
        "properties": { "order": ['range', 'speed'] }
        })
    
    range = quantity_with_display(
        type=float,
        unit='millimeter',
        description='Range of the linear dispensing pattern from the center.',
        a_eln=dict(component='NumberEditQuantity', default=5.0)
    )

    speed = quantity_with_display(
        type=float,
        description='Speed of the dispensing in the linear pattern.',
        a_eln=dict(component='NumberEditQuantity', minValue=0, maxValue=1.0, default=1.0)
    )

class DispenseProfile(MSection):
    '''
    A profile for dispensing materials in a process step.
    '''

    m_def = Section(a_eln={
        "properties": { "order": ['volume', 'delay', 'rate', 'height', 'pattern'] }
        })
    

    volume = quantity_with_display(
        type=float,
        unit='microliter',
        description='Total volume of material to be dispensed.',
        a_eln=dict(component='NumberEditQuantity', default=100.0)
    )

    delay = quantity_with_display(
        type=float,
        unit='second',
        description='Delay before spin coater start spining.',
        a_eln=dict(component='NumberEditQuantity', default=5.0)
    )

    rate = quantity_with_display(
        type=float,
        unit='microliter/second',
        description='Rate at which the material is dispensed.',
        a_eln=dict(component='NumberEditQuantity', default=10.0)
    )

    height = quantity_with_display(
        type=float,
        unit='millimeter',
        description='Height of the dispensing tip above the substrate.',
        a_eln=dict(component='NumberEditQuantity', default=5.0)
    )

    pattern = quantity_with_display(
        type=MEnum("Static", "Linear"),
        description='Pattern of dispensing',
        a_eln=dict(component='EnumEditQuantity', default="Static")
    )

    pattern_details = SubSection(
        section_def=LinearDispensePattern,
        description='Details of the linear dispensing pattern if selected.',
        a_eln=dict(component='SubSectionEditQuantity')
    )

class Quenching(MSection):
    '''
    A section for quenching in a process step.
    '''

    m_def = Section(a_eln={
        "properties": { "order": ['temperature', 'duration'] }
        })

    dispense_profile = SubSection(
        section_def=DispenseProfile,
        description='Profile for dispensing materials during the spin coating step.',
        a_eln=dict(component='SubSectionEditQuantity')
    )

    solvent_system = ELNSample(
        description='The solvent used in the quenching.',
        a_eln=dict(component='ReferenceEditQuantity')
    )

class SpinCoatingStep(ProcessStep):
    '''
    A process step for spin coating.
    '''

    name = quantity_with_display(
        type=MEnum("Spin Coating", "Spin Coating with Quenching"),
        description='Name of the spin coating step.',
        a_eln=dict(component='EnumEditQuantity', default='Spin Coating')
    )

    spin_duration = quantity_with_display(
        type=int,
        unit='second',
        description='Duration of the spin coating step in seconds.',
        a_eln=dict(component='NumberEditQuantity', default=60)
    )

    spin_ramp = SubSection(
        section_def=SpinRamp,
        description='Spin ramp profile for the spin coating step.',
        a_eln=dict(component='SubSectionEditQuantity')
    )

    dispense_profile = SubSection(
        section_def=DispenseProfile,
        description='Profile for dispensing materials during the spin coating step.',
        a_eln=dict(component='SubSectionEditQuantity')
    )

    solvent_system = ELNSample(
        description='The solvent used in the spin coating process.',
        a_eln=dict(component='ReferenceEditQuantity')
    )

    material_system = ELNSample(
        description='The material being coated in the spin coating process.',
        a_eln=dict(component='ReferenceEditQuantity')
    )

class TemperatureRamp(MSection):
    '''
    Temperature ramp profile for annealing.
    '''

    m_def = Section(
        a_plotly_graph_object=[{
            "data": { "x": "#time", "y": "#temperature"},
        }]
    )
    
    temperature = quantity_with_display(
        type=float,
        unit='degC',
        shape=["*"],
        description='Temperature at each step in the ramp.',
        a_eln=dict(component='NumberEditQuantity', default=[50.0, 100.0, 150.0])
    )

    time = quantity_with_display(
        type=int,
        unit='second',
        shape=["*"],
        description='Time at each temperature in the ramp.',
        a_eln=dict(component='NumberEditQuantity', default=[300, 600, 900])
    )

class HotplateAnnealingStep(ProcessStep):
    '''
    A process step for hotplate annealing.
    '''

    name = quantity_with_display(
        type=MEnum("Hotplate Annealing"),
        description='Name of the hotplate annealing step.',
        a_eln=dict(component='EnumEditQuantity', default='Hotplate Annealing')
    )

    temperature = quantity_with_display(
        type=float,
        unit='degC',
        description='Temperature of the hotplate during annealing.',
        a_eln=dict(component='NumberEditQuantity', default=100.0)
    )

    duration = quantity_with_display(
        type=int,
        unit='second',
        description='Duration of the hotplate annealing step in seconds.',
        a_eln=dict(component='NumberEditQuantity', default=600)
    )

    temperature_ramp = SubSection(
        section_def=TemperatureRamp,
        description='Temperature ramp profile for the hotplate annealing step.',
        a_eln=dict(component='SubSectionEditQuantity')
    )


In [4]:
class ThinFilmSynthesisProcess(ELNProcess):
    """
    A process for synthesizing thin films.

    samples:
        - ENL sample
        
    """

    composite_system = SubSection(
        section_def=CompositeSystemReference,
        a_eln=dict(section_type='composite_system')
    )

    instrument = SubSection(
        section_def=InstrumentReference,
        a_eln=dict(section_type='instrument')
    )

In [ ]:

class ThinFilmSynthesisExperiment(ELNExperiment):
    """
    A class to represent a thin film synthesis experiment.

    steps:
        - ProcessStep
        - MeasurementStep
    """
    m_def = Section(
        a_eln=dict(section_type='experiment', title='Thin Film Synthesis Experiment')
    )

    steps = SubSection(
        section_def=ThinFilmSynthesisProcess,
        a_eln=dict(section_type='process', title='Thin Film Synthesis Process')
    )

    

In [6]:
from nomad.metainfo import SchemaPackage
from nomad.datamodel import EntryArchive
import yaml

def create_schema_package():
    return EntryArchive(
        definitions=SchemaPackage(
            name='thin_film_synthesis',
            description='Schema package for thin film synthesis processes and experiments.',
            sections=[
                SpinRamp.m_def,
                LinearDispensePattern.m_def,
                DispenseProfile.m_def,
                Quenching.m_def,
                SpinCoatingStep.m_def,
                TemperatureRamp.m_def,
                HotplateAnnealingStep.m_def,
                ThinFilmSynthesisProcess.m_def,
                ThinFilmSynthesisExperiment.m_def,
                
            ]
        )
    )

def save_schema_package_to_yaml():
    with open('thin_film_synthesis_schema_package.archive.yaml', 'wt') as f:
        f.write(yaml.dump(create_schema_package().m_to_dict(with_out_meta=True), indent=2))

save_schema_package_to_yaml()
print(yaml.dump(create_schema_package().m_to_dict(with_out_meta=True), indent=2))

definitions:
  description: Schema package for thin film synthesis processes and experiments.
  name: thin_film_synthesis
  section_definitions:
  - description: Spin ramp profile for spin coating.
    m_annotations:
      plotly_graph_object:
      - data:
          x: '#time'
          y: '#speed'
    name: SpinRamp
    quantities:
    - description: Speed of the spin ramp in revolutions per minute.
      m_annotations:
        display:
        - unit: rpm
        eln:
        - component: NumberEditQuantity
          default:
          - 1000
          - 2000
          - 3000
      name: speed
      shape:
      - '*'
      type:
        type_data: int
        type_kind: python
      unit: revolutions_per_minute
    - description: Time at each speed in the spin ramp.
      m_annotations:
        display:
        - unit: second
        eln:
        - component: NumberEditQuantity
          default:
          - 10
          - 30
          - 60
      name: time
      shape:
      - '*'

In [3]:

class TipType(MEnum):
    m_def = Section(a_enum=True)
    high_volume = 0
    low_volume = 1
    filtered = 2

class PipettePreparationSection(MSection):
    """Represents pipette preparation for dispensing solutions."""
    
    m_def = Section()
    
    pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used for dispensing.'
    )
    
    tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used.'
    )
    
    solution_number = Quantity(
        type=int,
        description='Solution number to be dispensed.'
    )
    
    solution_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume of solution to be dispensed.'
    )
    
    solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for the solution.'
    )

class MixingSection(MSection):
    """Represents mixing parameters for solutions."""
    
    m_def = Section()
    
    mixing_cycles_aspiration = Quantity(
        type=int,
        description='Number of mixing cycles for aspiration (0-9).'
    )
    
    liquid_class_index_aspiration = Quantity(
        type=int,
        description='Liquid class index for aspiration.'
    )
    
    liquid_class_index_dispensing = Quantity(
        type=int,
        description='Liquid class index for dispensing.'
    )

class SpinParameters(MSection):
    """Common spin coating parameters."""
    
    m_def = Section()
    
    spin_acceleration = Quantity(
        type=float,
        unit='rpm/s',
        description='Acceleration rate of spin coater.'
    )
    
    spin_time = Quantity(
        type=float,
        unit='s',
        description='Duration of spinning.'
    )
    
    spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Velocity of spin coater.'
    )
    
    spincoater_number = Quantity(
        type=int,
        description='Spincoater device number.'
    )

class AnnealingParameters(MSection):
    """Annealing parameters section."""
    
    m_def = Section()
    
    annealing_station_number = Quantity(
        type=int,
        description='Annealing station number.'
    )
    
    annealing_temperature = Quantity(
        type=float,
        unit='°C',
        description='Temperature for annealing.'
    )
    
    annealing_time = Quantity(
        type=float,
        unit='s',
        description='Duration of annealing.'
    )

class QuenchParameters(MSection):
    """Parameters for quenching step in spin coating."""
    
    m_def = Section()
    
    quench_pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used for quenching.'
    )
    
    quench_tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used for quenching.'
    )
    
    quench_solution_number = Quantity(
        type=int,
        description='Quench solution number.'
    )
    
    quench_solution_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume of quench solution.'
    )
    
    quench_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for quench solution.'
    )
    
    quench_height_above_substrate = Quantity(
        type=float,
        unit='mm',
        description='Height above substrate for quench dispensing.'
    )
    
    quench_liquid_class_index_aspiration = Quantity(
        type=int,
        description='Liquid class index for quench aspiration.'
    )
    
    quench_liquid_class_index_dispensing = Quantity(
        type=int,
        description='Liquid class index for quench dispensing.'
    )
    
    quench_spin_delay = Quantity(
        type=float,
        unit='s',
        description='Delay between quenching and second spin.'
    )

class SecondSpinParameters(MSection):
    """Parameters for second spin after quenching."""
    
    m_def = Section()
    
    second_spin_acceleration = Quantity(
        type=float,
        unit='rpm/s',
        description='Acceleration rate of second spin.'
    )
    
    second_spin_time = Quantity(
        type=float,
        unit='s',
        description='Duration of second spin.'
    )
    
    second_spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Velocity of second spin.'
    )

class SpinCoatingStep(ProcessStep):
    """Base class for spin coating process steps."""
    
    m_def = Section()
    
    precursor = Quantity(
        type=Reference(CompositeSystemReference),
        description='Reference to the precursor material.'
    )
    
    solvent = Quantity(
        type=Reference(CompositeSystemReference),
        description='Reference to the solvent used.'
    )
    
    pipette_preparation = SubSection(
        section_def=PipettePreparationSection,
        description='Pipette preparation parameters.'
    )
    
    mixing = SubSection(
        section_def=MixingSection,
        description='Solution mixing parameters.',
        repeats=False,
        required=False
    )
    
    pipette_dispensing_height = Quantity(
        type=float,
        unit='mm',
        description='Height above substrate for dispensing.'
    )
    
    dispense_spin_blade_delay = Quantity(
        type=float,
        unit='s',
        description='Delay between dispensing and spin/blade operation.'
    )
    
    spin_parameters = SubSection(
        section_def=SpinParameters,
        description='Spin coating parameters.'
    )
    
    put_on_hotplate = Quantity(
        type=bool,
        description='Whether to put sample on hotplate after spinning.',
        required=False
    )
    
    annealing = SubSection(
        section_def=AnnealingParameters,
        description='Annealing parameters.',
        repeats=False,
        required=False
    )

class WhirlSpinCoatingStep(SpinCoatingStep):
    """Whirl spin coating process, with spinning before dispensing."""
    
    m_def = Section()
    
    initial_spin_velocity = Quantity(
        type=float,
        unit='rpm',
        description='Initial spin velocity before dispensing.'
    )
    
    arm_speed_while_dispensing = Quantity(
        type=float,
        unit='%',
        description='Arm speed while dispensing (0-100%).'
    )
    
    dispense_range = Quantity(
        type=float,
        unit='mm',
        description='Range for dispensing along arm path.'
    )

class PipetteQuenchingStep(SpinCoatingStep):
    """Spin coating with quenching step."""
    
    m_def = Section()
    
    quench_parameters = SubSection(
        section_def=QuenchParameters,
        description='Parameters for quenching step.'
    )
    
    second_spin = SubSection(
        section_def=SecondSpinParameters,
        description='Parameters for second spin after quenching.'
    )
    
    decap_and_recap_vial = Quantity(
        type=bool,
        description='Whether to decap and recap vial after process.',
        required=False
    )

class SolventMixingStep(ProcessStep):
    """Solvent mixing process step."""
    
    m_def = Section()
    
    aspirate_index = Quantity(
        type=int,
        description='Index for aspiration.'
    )
    
    aspirate_solution_number = Quantity(
        type=int,
        description='Solution number for aspiration.'
    )
    
    aspirate_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for aspiration.'
    )
    
    aspirate_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume to aspirate.'
    )
    
    dispense_index = Quantity(
        type=int,
        description='Index for dispensing.'
    )
    
    dispense_solution_number = Quantity(
        type=int,
        description='Solution number for dispensing.'
    )
    
    dispense_solvent_station_number = Quantity(
        type=int,
        description='Solvent station number for dispensing.'
    )
    
    dispense_volume = Quantity(
        type=float,
        unit='uL',
        description='Volume to dispense.'
    )
    
    load_dispense_matrix = Quantity(
        type=bool,
        description='Whether to load dispense matrix.',
        required=False
    )
    
    pipette_rack_number = Quantity(
        type=int,
        description='Pipette rack number used.'
    )
    
    tip_type = Quantity(
        type=TipType,
        description='Type of pipette tip used.'
    )

class TakePhotoStep(ProcessStep):
    """Step to take a photo of the substrate."""
    
    m_def = Section()
    
    take_photo = Quantity(
        type=bool,
        description='Whether to take a photo.',
        default=True)
class PerovskiteFilmSynthesis(SynthesisMethod):
    """Complete perovskite film synthesis method."""
    
    m_def = Section()
    
    instruments = Quantity(
        type=Reference(InstrumentReference),
        shape=['*'],
        description='References to instruments used in synthesis.'
    )
    
    samples = Quantity(
        type=Reference(CompositeSystemReference),
        shape=['*'],
        description='References to samples produced.'
    )
    
    steps = SubSection(
        section_def=ProcessStep,
        description='Process steps in the synthesis.',
        repeats=True
    )

# Register the schema
if __name__ == "__main__":
    from nomad.datamodel import Schema
    schema = Schema(name='perovskite_synthesis', 
                   section_defs=[
                       TipType, PipettePreparationSection, MixingSection, 
                       SpinParameters, AnnealingParameters, QuenchParameters,
                       SecondSpinParameters, SpinCoatingStep, WhirlSpinCoatingStep,
                       PipetteQuenchingStep, SolventMixingStep, TakePhotoStep,
                       PerovskiteFilmSynthesis
                   ])
    description='Process steps in the synthesis.',
    repeats=True
    

AttributeError: 'NoneType' object has no attribute 'evaluate'